In [24]:
import functools

import numpy as np
import tensorflow as tf

In [25]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)
np.set_printoptions(precision=3, suppress=True)


In [26]:
#значение которой вы собиратесь предсказывать с помощью модели
LABEL_COLUMN = 'survived'
LABELS = [0, 1]


In [27]:
# Сейчас прочитайте данные CSV из файла и создайте датасет.
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Значение искусственно занижено для удобства восприятия.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [28]:
# Каждый элемент в датасете это пакет представленный в виде кортежа (много примеров, много меток). 
# Данные из примеров организованы в тензоры столбцы (а не тензоры строки),
# каждый с таким количеством элементов каков размер пакета (12 в этом случае).
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [29]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'male' b'female' b'male']
age                 : [27. 36. 28. 19. 28.]
n_siblings_spouses  : [1 0 0 0 8]
parch               : [0 0 0 0 2]
fare                : [ 21.    135.633  10.5    26.     69.55 ]
class               : [b'Second' b'First' b'Second' b'Second' b'Third']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'n']


In [30]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'male' b'female' b'male' b'male']
age                 : [28. 28. 16. 28.  9.]
n_siblings_spouses  : [3 0 0 0 4]
parch               : [1 0 0 0 2]
fare                : [25.467  7.896  7.75   7.896 31.388]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Queenstown' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'n']


In [31]:
# Если ваши данные уже имеют подходящий числовой формат, 
# вы можете упаковать данные в вектор, прежде чем передать их в модель:
# CSV файл может содержать множество видов данных. 
# Обычно, перед тем как передать данные в вашу модель, вы хотите преобразовать эти смешанные типы в вектор фиксированной длины.
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [42. 28. 34. 32. 48.]
n_siblings_spouses  : [1. 0. 0. 0. 1.]
parch               : [0. 0. 0. 0. 0.]
fare                : [27.    8.05 13.   13.   39.6 ]


In [32]:
example_batch, labels_batch = next(iter(temp_dataset)) 


In [33]:
# Вот простая функция которая упакует вместе все колонки:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [34]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[28.     0.     0.     7.75 ]
 [ 7.     4.     1.    39.688]
 [21.     0.     0.     8.663]
 [28.     0.     0.    15.5  ]
 [28.     3.     1.    25.467]]

[0 0 0 0 0]


In [35]:
show_batch(raw_train_data)


sex                 : [b'female' b'male' b'male' b'female' b'male']
age                 : [35. 59. 28. 28. 28.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [83.475 13.5    8.05   7.733 31.   ]
class               : [b'First' b'Second' b'Third' b'Third' b'First']
deck                : [b'C' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Queenstown' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [36]:
example_batch, labels_batch = next(iter(temp_dataset)) 


In [37]:
# Так что выберите более общий препроцессор который выбирает список числовых свойств
# и упаковывает их в одну колонку:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_freatures = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [38]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [39]:
show_batch(packed_train_data)


sex                 : [b'male' b'female' b'female' b'female' b'male']
class               : [b'Third' b'Second' b'First' b'First' b'First']
deck                : [b'unknown' b'F' b'E' b'unknown' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'n']
numeric             : [[ 44.      0.      1.     16.1  ]
 [ 34.      0.      0.     10.5  ]
 [ 30.      0.      0.     56.929]
 [ 22.      0.      0.    151.55 ]
 [ 36.      1.      2.    120.   ]]


In [40]:
example_batch, labels_batch = next(iter(packed_train_data)) 


In [41]:
# Непрерывные данные должны быть всегда нормализованы.

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [42]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])



In [43]:
def normalize_numeric_data(data, mean, std):
  # Центрируем данные
  return (data-mean)/std


In [44]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000001609A6F30D0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [45]:
# Когда вы обучаете модель добавьте этот столбец признаков чтобы выбрать и центрировать блок числовых данных:
example_batch['numeric']


<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[24.   ,  2.   ,  0.   , 24.15 ],
       [29.   ,  0.   ,  4.   , 21.075],
       [34.   ,  0.   ,  1.   , 23.   ],
       [32.   ,  0.   ,  0.   ,  7.925],
       [45.   ,  0.   ,  1.   , 14.454]], dtype=float32)>

In [46]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.45 ,  1.264, -0.479, -0.187],
       [-0.05 , -0.474,  4.565, -0.244],
       [ 0.349, -0.474,  0.782, -0.209],
       [ 0.189, -0.474, -0.479, -0.485],
       [ 1.228, -0.474,  0.782, -0.365]], dtype=float32)

In [47]:
# Некоторые из столбцов в данных CSV являются категорийными. 
# Это значит, что содержимое является одним из ограниченного числа вариантов.
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [48]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [49]:
categorical_columns


[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [50]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])


[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [51]:
# Добавьте две коллекции столбцов признаков и передайте их в tf.keras.layers.DenseFeatures 
# чтобы создать входной слой который извлечет и предобработает оба входных типа:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)


In [52]:
print(preprocessing_layer(example_batch).numpy()[0])


[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.45   1.264
 -0.479 -0.187  1.     0.   ]


In [53]:
# Постройте tf.keras.Sequential начиная с препроцессингового слоя preprocessing_layer.
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [54]:
# Теперь модель может быть реализована и обучена.
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [55]:
# Обучаем модель
model.fit(train_data, epochs=20)


Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 0s 2ms/step - loss: 0.5200 - accuracy: 0.7416
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4229 - accuracy: 0.8198
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4106 - accuracy: 0.8309
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3876 - accuracy: 0.8421
Epoch 5/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3803 - accuracy: 0.8357
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3752 - accuracy: 0.8373
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3616 - accuracy: 0.8421
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3632 - accuracy: 0.8517
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3553 - accurac

In [56]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 2ms/step - loss: 0.4493 - accuracy: 0.8409


Test Loss 0.4493350386619568, Test Accuracy 0.8409090638160706


In [57]:
predictions = model.predict(test_data)

# Покажем некоторые результаты
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Consider rewriting this model with the Functional API.
Predicted survival: 95.46%  | Actual outcome:  SURVIVED
Predicted survival: 35.27%  | Actual outcome:  SURVIVED
Predicted survival: 7.23%  | Actual outcome:  DIED
Predicted survival: 99.97%  | Actual outcome:  SURVIVED
Predicted survival: 14.62%  | Actual outcome:  DIED
